In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
IMAGE_PATH = "training_data"
BATCH_SIZE = 50
EPOCH = 50
TRAINING_SIZE = 0.8

### Data preperation

In [36]:
size = 256,128

def get_image(file_path):
    image = Image.open(file_path).convert('RGB')
    image.thumbnail(size, Image.ANTIALIAS)
    return image
        

file_path = "./meta-data/train.csv"
meta_data = pd.read_csv(file_path)

meta_data.describe()
training_data = []



for i in range(len(meta_data)):    
    image = get_image(IMAGE_PATH+"/"+meta_data.iloc[i][0])
    training_data.append([np.array(image),meta_data.iloc[i][1:85]])
    if len(training_data) % 1260 == 0:
        print("saving data @ {}".format(i+1))
        np.save("training_data_{}.npy".format(i),training_data)
        training_data = []
    
    
    

saving data @ 1260
saving data @ 2520
saving data @ 3780
saving data @ 5040
saving data @ 6300
saving data @ 7560
saving data @ 8820
saving data @ 10080
saving data @ 11340
saving data @ 12600


### Load training data           

In [39]:
X_train = []
Y_train = []
for i in range(1,11):
    training_data = np.load("training_data_{}.npy".format((i*1260)-1))
    print(i)
    for data in training_data:
        X_train.append(data[0])
        Y_train.append(data[1])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(X_train.size)

1
2
3
4
5
6
7
8
9
10
12600


## Build Tensorflow model

In [40]:
def weight_variable(name, shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.get_variable(name, initializer=initial)

def bias_variable(name, shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.get_variable(name, initializer=initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

In [49]:
PIXEL_COUNT = X_train[0].shape[1] * X_train[0].shape[2]
LABEL_COUNT = Y_train[0].shape[0]
# Create placeholders for image data and expected point positions

class Model(object):
    xxx = 0
    
# Build neural network
def build_model():
    x_placeholder = tf.placeholder(tf.float32, shape=[None, PIXEL_COUNT])
    y_placeholder = tf.placeholder(tf.float32, shape=[None, LABEL_COUNT])

    x_image = tf.reshape(x_placeholder, [-1, 256, 128, 3])
    # Convolution Layer 1
    W_conv1 = weight_variable("w1", [5, 5, 1, 32])
    b_conv1 = bias_variable("b1", [32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    # Convolution Layer 2
    W_conv2 = weight_variable("w2", [2, 2, 32, 64])
    b_conv2 = bias_variable("b2", [64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    # Convolution Layer 3
    W_conv3 = weight_variable("w3", [2, 2, 64, 128])
    b_conv3 = bias_variable("b3", [128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    h_pool3 = max_pool_2x2(h_conv3)
    # Dense layer 1
    h_pool3_flat = tf.reshape(h_pool3, [-1, 8*16*128])
    W_fc1 = weight_variable("w4", [8*16*128, 500])
    b_fc1 = bias_variable("b4", [500])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)   
    # Dense layer 2
    W_fc2 = weight_variable("w5", [500, 500])
    b_fc2 = bias_variable("b5", [500])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)  
    # Output layer
    W_out = weight_variable("w6", [500, LABEL_COUNT])
    b_out = bias_variable("b6", [LABEL_COUNT])
    
    output = tf.matmul(h_fc2, W_out) + b_out
    
    model = Model()
    model.x_placeholder = x_placeholder
    model.y_placeholder = y_placeholder
    model.output = output
    
    return model